In [2]:
import requests
import math
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint
from datetime import datetime

ModuleNotFoundError: No module named 'matplotlib'

This API provides access to the JPL/SSD small-body mission design suite. The following operation modes are available:

* Mode A (accessible) - retrieves the list of accessible small bodies based on user-defined constraint.
* Mode Q (query) - retrieves pre-computed mission options to a specific object. Both impulsive and low-thrust gravity-assist mission options are available.
* Mode M (map) - an extension of mode Q for the impulsive case, returns the data required to render a porkchop plot with multiple parameters.
* Mode T (mission extension) - retrieves the list of small bodies that come closest (or within a prescribed distance) to a user-defined orbit during a certain period of time. This is a crude filter for finding potential candidates for mission extensions.

In [3]:
url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
asteroid_name = '2012TC4' # designation (provisional or IAU-number) of the desired object (e.g., 2015 AB or 141P or 433).
                         # NOTE: when submitting a des containing a space in your query string, you must 
                         # replace the space with %20, for example 2015%20AB.
mjd0 = 59215            # 21Jan2021 # first launch date to be explored (Modified Julian Date)
span = 365                # duration of the launch-date period to be explored (days)
tof_min = 50          # minimum time of flight to be considered (days)
tof_max = 720       # maximum time of flight to be considered (days)
step = 5            # time step used to advance both the launch date and the time of flight (days). 
                    # The size of the transfer map is limited to 1,500,000 points

sim_lim_points = 1500000 #1.5 millions
if int(span)/int(step) > sim_lim_points:
    print('outside of tool limits') 

url = f'{url_base}?sstr={str(asteroid_name)}&mjd0={str(mjd0)}&span={str(span)}&tof-min={str(tof_min)}&tof-max={str(tof_max)}&step={str(step)}'
r = requests.get(url)

In [4]:
data = r.json()

print(data)

{'signature': {'source': 'NASA/JPL Small-Body Mission Design API', 'version': '1.1'}, 'md_constraints': {'sstr': '2012TC4', 'mjd0': '59215', 'tof-min': '50', 'tof-max': '720', 'step': '5', 'span': '365'}, 'object': {'data_arc': '1897', 'md_orbit_id': '61', 'orbit_class': 'APO', 'spkid': '3609693', 'condition_code': '1', 'orbit_id': '61', 'fullname': '(2012 TC4)', 'des': '2012 TC4'}, 'modified': '2019-07-29 19:57:45 PDT', 'dv_lowthrust': {'sep': 9.09526, 'const': 7.41117}, 'fields': ['MJD0', 'MJDf', 'vinf_dep', 'vinf_arr', 'phase_ang', 'earth_dist', 'elong_arr', 'decl_dep', 'approach_ang', 'tof'], 'selectedMissions': [[59934, 60134, 12.2454, 18.4252, 5.39, 0.8832, 154.78, -7.91, 114.67, 200], [61479, 61589, 12.2182, 26.6691, 7.39, 1.1681, 139.92, -23.98, 100.93, 110], [60764, 60864, 12.1803, 27.2703, 7.06, 1.0408, 143.22, -24.72, 103.76, 100], [58804, 58839, 12.1714, 17.882, 121.78, 0.236, 88.09, -0.33, 156.62, 35], [62204, 62319, 12.1563, 25.2982, 7.14, 1.3043, 131.96, -21.96, 98.66, 1

In [5]:
data.keys()

dict_keys(['signature', 'md_constraints', 'object', 'modified', 'dv_lowthrust', 'fields', 'selectedMissions', 'tof', 'dep_date', 'vinf_dep', 'vinf_arr', 'phase_ang', 'earth_dist', 'elong_arr', 'decl_dep', 'approach_ang'])

In [6]:
pprint(data['fields'])

Pretty printing has been turned OFF


In [7]:
available_missions = len(data['selectedMissions'])
mission_profiles={};
for mission_id in range(0,available_missions-1):
    sel_profile={};
    sel_profile={"fullname": data["object"]["fullname"],
              "mjd0": data["selectedMissions"][mission_id][0],
              "mjdf": data["selectedMissions"][mission_id][1],
              "tof": data["selectedMissions"][mission_id][9],
              "vinf_dep": data["selectedMissions"][mission_id][2],
              "vinf_arr": data["selectedMissions"][mission_id][3],
              "earth_dist": data["selectedMissions"][mission_id][5],
              "phase_ang": data["selectedMissions"][mission_id][4], 
              "elong_arr": data["selectedMissions"][mission_id][6], 
              "decl_dep": data["selectedMissions"][mission_id][7],
              "approach_ang": data["selectedMissions"][mission_id][8],      
             };
    mission_profiles[mission_id]=sel_profile;

In [8]:
def get_mission_profiles(asteroid_name,mjd0,span,tof_min,tof_max,step):
    # asteroid_name:    designation (provisional or IAU-number) of the desired object (e.g., 2015 AB or 141P or 433).
    #                   NOTE: when submitting a des containing a space in your query string, you must replace the space with %20, for example 2015%20AB
    # mjd0:             first launch date to be explored (Modified Julian Date)
    # span:             duration of the launch-date period to be explored (days)
    # tof-min:          minimum time of flight to be considered (days)
    # tof-max:          maximum time of flight to be considered (days)
    # step:             time step used to advance both the launch date and the time of flight (days). 
                        
    # The size of the transfer map is limited to 1,500,000 points
    sim_lim_points = 1500000 #1.5 millions
    if int(span)/int(step) > sim_lim_points:
        print('outside of tool limits') # TODO return error
    
    # Construction of the HTTP request
    url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
    url = f'{url_base}?sstr={str(asteroid_name)}&mjd0={str(mjd0)}&span={str(span)}&tof-min={str(tof_min)}&tof-max={str(tof_max)}&step={str(step)}'
    r = requests.get(url)
    data = r.json()
    
    # Elaboration of data
    available_missions = len(data['selectedMissions'])
    mission_profiles={};
    for mission_id in range(available_missions):
        sel_profile={"fullname": data["object"]["fullname"],
                  "mjd0": data["selectedMissions"][mission_id][0],
                  "mjdf": data["selectedMissions"][mission_id][1],
                  "tof": data["selectedMissions"][mission_id][9],
                  "vinf_dep": data["selectedMissions"][mission_id][2],
                  "vinf_arr": data["selectedMissions"][mission_id][3],
                  "earth_dist": data["selectedMissions"][mission_id][5],
                  "phase_ang": data["selectedMissions"][mission_id][4], 
                  "elong_arr": data["selectedMissions"][mission_id][6], 
                  "decl_dep": data["selectedMissions"][mission_id][7],
                  "approach_ang": data["selectedMissions"][mission_id][8],      
                 };
        mission_profiles[mission_id]=sel_profile;
    
    # calcoli porkchop
    
    return mission_profiles

In [9]:
missions_2012tc4 = get_mission_profiles('2012TC4',59215,365,50,700,5)

In [ ]:
missions_2012tc4

In [ ]:
dv=[]
for profile in missions_2012tc4:
    #dv[profile] = missions_2012tc4[profile]['vinf_arr'] - missions_2012tc4[profile]['vinf_dep']

In [ ]:
print(dv)

In [ ]:
url = 'https://ssd-api.jpl.nasa.gov/mdesign.api?sstr=65803&ltdata=true'
r = requests.get(url)
data = r.json()
pprint(data)

* Mode M - In addition to querying the database like in mode Q (ballistic), compute all ballistic mission options to the specified target within certain ranges of launch dates and times of flight.
    * In addition, the values of the x-_y_ axes of the maps are also provided:
        * dep_date - departure dates from Earth (Modified Julian Date), corresponding to the x-axis.
        * tof - times of flight to the target (days), corresponding to the y-axis.
        * If dep_date has m elements and tof has n, then the 2D arrays are of dimension n x m.
        * vinf_dep
        * vinf_arr

In [10]:
def get_mission_porkchop(asteroid_name,mjd0,span,tof_min,tof_max,step):
    # asteroid_name:    designation (provisional or IAU-number) of the desired object (e.g., 2015 AB or 141P or 433).
    #                   NOTE: when submitting a des containing a space in your query string, you must replace the space with %20, for example 2015%20AB
    # mjd0:             first launch date to be explored (Modified Julian Date)
    # span:             duration of the launch-date period to be explored (days)
    # tof-min:          minimum time of flight to be considered (days)
    # tof-max:          maximum time of flight to be considered (days)
    # step:             time step used to advance both the launch date and the time of flight (days). 
                        
    # The size of the transfer map is limited to 1,500,000 points
    sim_lim_points = 1500000 #1.5 millions
    if int(span)/int(step) > sim_lim_points:
        print('outside of tool limits') 
    
    # Construction of the HTTP request
    url_base = 'https://ssd-api.jpl.nasa.gov/mdesign.api'
    url = f'{url_base}?sstr={str(asteroid_name)}&mjd0={str(mjd0)}&span={str(span)}&tof-min={str(tof_min)}&tof-max={str(tof_max)}&step={str(step)}'
    r = requests.get(url)
    data = r.json()
    
    # Elaboration of data
    m = len(data["dep_date"])
    n = len(data["tof"])
    porkchop_map=[[0]*m]*n;
    #porkchop_map = np.zeros([n,m]) # porkchop_map[i,j]
    for i in range(n):
        for j in range(m):
            porkchop_map[i][j]=data["vinf_arr"][i][j]-data["vinf_dep"][i][j]
    return porkchop_map

In [11]:
missions_2012tc4_pc = get_mission_porkchop('2012TC4',59215,365,50,700,5)


In [ ]:
len(data["vinf_arr"])


In [ ]:
len(missions_2012tc4_pc[1]), len(missions_2012tc4_pc)

In [ ]:
n = len(data["tof"])
m = len(data["dep_date"])
m,n

In [ ]:
tof = data["tof"]
dep_date = data["dep_date"]

In [ ]:
tof

In [12]:
plt.contour(dep_date, tof, missions_2012tc4_pc)

NameError: name 'plt' is not defined